In [1]:
import re
import numpy as np
import pandas as pd

# Computer Vision
import cv2
import pytesseract
from pytesseract import Output

In [2]:
def image_read(filename):
    image = cv2.imread(filename)
    return image


def image_2_text(image):
    box_image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    lines = {}
    count = 0

    boxes = pytesseract.image_to_data(
        gray, config='--psm {}'.format(6), lang='eng', output_type=Output.DICT)

    prev = 0
    words = []

    for i, j, m, e, k in zip(boxes['text'], boxes['left'], boxes['top'], boxes['width'], boxes['conf']):
        if (j-prev) > 30:
            if '|' not in i and '——' not in i:
                words.append([m, j, i])
        else:
            try:
                if '|' not in i and '——' not in i:
                    words[-1][-1] += ' '+i
            except:
                if '|' not in i and '——' not in i:
                    words = [[m, j, i]]
        if '|' not in i and '——' not in i:
            prev = j+e
        if k == str(-1):
            prev = 0
            lines[count] = words
            count += 1
            words = []
    return lines


def column_format(lines):
    columns = []
    thres = [69, 335, 600, 865]

    for i, j in lines.items():
        column = []
        count = 0
        for n in thres:
            try:
                if abs(j[count][1]-n) < 20:
                    column.append(j[count])
                    count += 1
                else:
                    column.append([])
            except:
                column.append([])
        columns.append(column)

    return columns


def data_group(columns, val=5):
    companies_profile = []
    temp = []

    for j in range(4):
        # Same Column
        company_profile = []
        for i in columns[val:]:
            if len(i[j]):
                company_profile.append(i[j][-1])
            else:
                # Line Break
                companies_profile.append(company_profile)
                company_profile = []
        if len(company_profile):
            companies_profile.append(company_profile)

    return companies_profile


def data_field(companies_profile):
    companies_data = []

    for company_profile in companies_profile:
        if len(company_profile):
            flag = {'company': 1, 'address': 1, 'email': 1, 'name': 1}
            company_data = {'company': '', 'address': '',
                            'email': '', 'name': ''}
            for k in company_profile:
                if flag['company'] and len(re.findall(r'[0-9]', k)) == 0:
                    company_data['company'] += ' '+k
                elif flag['address'] and len(re.findall(r'[0-9]', k)):
                    flag['company'] = 0
                    company_data['address'] += ' '+k
                elif '@' in k or '.com' in k or 'org' in k:
                    flag['address'] = 0
                    company_data['email'] += k
                else:
                    company_data['name'] += ' '+k
            company_data = {m: n.strip() for m, n in company_data.items()}
            companies_data.append(company_data)

    df = pd.DataFrame(companies_data)

    return df


def text_formatting(lines):
    columns = column_format(lines)
    companies_profile = data_group(columns)
    df = data_field(companies_profile)
    return df

In [3]:
image = image_read('static/companies.jpeg')
lines = image_2_text(image)
df = text_formatting(lines)

In [4]:
df.head()

,company,address,email,name
0,Association of Marion Landing Owners,"8457 SW 65th Ave Rd Ocala, FL 34476 352.237.72...",alewis@lelandmanagement.com,Adira Lewis
1,"Atlantic Beach, City of","800 Seminole Road Atlantic Beach, FL 32233 904...",hmcnally@coab.us,Harry McNally
2,"Auburn Water System, Inc.","3097 Locke Lane Crestview, FL 32536 850.682.34...",,Doug Sims
3,"Auburndale, City of","PO Box 186 Auburndale, FL 33823 863.965.5549 T...",njay@auburndalefl.com,Norris Jay
4,Aucilla Christian Academy,"7803 Aucilla Rd Monticello, FL 32344 850.997.3...",rfinlayson@aucilla.org,Richard Finlayson
